# Use pre-trained BERT model to predict NER labels


see `CAVEAT` cell below
- This cell will download a large (400+MB) pre-trained model and likely take several minutes to do so.
- The Bert model will be saved to the local filesystem and not downloaded on subsequent invocations. 

In [1]:
import time
ts = time.time()

In [10]:
%pip install datasets
%pip install evaluate
%pip install transformers
%pip install seqeval

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=b1d42997f4ef42791b067d0a163f045345af77d449348bb3baad40124f3c0d47
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
Note: you may need to restart the kernel to use updated packages.


In [3]:
from utils import *

In [4]:
# import needed dependences
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification
from transformers.pipelines.token_classification import TokenClassificationPipeline

In [5]:
# CAVEAT: Long-running cell on initial invocation
# load pre-trained BERT base cased model
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
bert_model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
bert_nlp: TokenClassificationPipeline = pipeline("ner", model=bert_model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
def annotate_bert(result: int) -> str:
    """ Helper function to tranlate NER int to class label """
    if len(result) == 0:
        return 'O'
    return result[0]['entity']

def predict_ner_bert(bert_nlp: TokenClassificationPipeline, labeled_dataset: Dataset) -> Tuple[list[list[str]], list[list[str]]]:
    """ Run inference on the tokens using trained BERT model """
    references: list[list[str]] = []
    predictions: list[list[str]] = []

    for row in tqdm(labeled_dataset, desc=str(len(labeled_dataset))):
        # add ground truth labels to references
        references.append([tag_names[id] for id in row['ner_tags']])
        # recognize named entity in a test tokens
        ner_results = bert_nlp(row['tokens'])
        # translate numerical index to NER class label
        predicted_tags = [annotate_bert(x) for x in ner_results]
        predictions.append(predicted_tags)
    return references, predictions


In [7]:
# Generate a list of ground truth NER labels and predictions
references, bert_predictions = predict_ner_bert(bert_nlp, test)

3453: 100%|██████████| 3453/3453 [1:04:22<00:00,  1.12s/it]


In [8]:
# Save NER results to disk
bert_path = os.path.join(interim_dir, "ner_results_bert.json")
save_ner_results(bert_path, references, bert_predictions)

# Load persisted NER results
# references1, predictions1 = load_ner_results(bert_path)


Saving NER results to ../data/interim/ner_results_bert.json


In [11]:
# Evaluate results
results = evaluate_results(references, bert_predictions)

bert_evaluation_path = os.path.join(interim_dir, "evaluation_results_bert.json")
save_evaluation_results(bert_evaluation_path, results)


Saving evaluation results to ../data/interim/evaluation_results_bert.json


In [13]:
te = time.time()
duration = te-ts
duration = float(f"{duration:.2f}")
print(f"Total running time: {duration} sec ")

Total running time: 3947.72 sec 
